# libtmux - tmux を制御するための Python スクリプトライブラリ

## インストール

In [2]:
!pip install libtmux

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.5/47.5 kB 597.4 kB/s eta 0:00:001m547.3 kB/s eta 0:00:01


## 新しいセッションの開始

In [53]:
import libtmux

TMUX_SESSION_NAME = 'Fabric'

def get_session(session_name):
    server = libtmux.Server()
    return server.sessions.filter(
        lambda x: x.session_name.startswith(session_name))

def new_session(session_name):
    server = libtmux.Server()
    sessions = get_session(session_name)
    if len(sessions) == 0:
        session = server.new_session(session_name=session_name)
        #session.rename_session('Fabric')
    else:
        session = sessions[0]
    
    return session

new_session(TMUX_SESSION_NAME)

Session($1 Fabric)

## セッション一覧

In [54]:
import libtmux

server = libtmux.Server()
print(f'Tmux sessions: {server.sessions}')
print(f'Tmux sessions[0]: {server.sessions[0]}')

Tmux sessions: [Session($4 4), Session($5 5), Session($6 6), Session($1 Fabric), Session($2 Fabric1), Session($0 JupyterLab)]
Tmux sessions[0]: Session($4 4)


In [55]:
!tmux list-sessions

4: 1 windows (created Sun Apr 23 05:36:20 2023)
5: 1 windows (created Sun Apr 23 05:44:53 2023)
6: 1 windows (created Sun Apr 23 05:47:03 2023)
Fabric: 1 windows (created Mon Apr 17 13:37:27 2023) (attached)
Fabric1: 1 windows (created Sat Apr 22 05:49:21 2023)
JupyterLab: 1 windows (created Fri Apr  7 08:54:56 2023)


## セッション名の変更

In [7]:
server.sessions[0].rename_session('JupyterLab')

Session($0 JupyterLab)

In [8]:
import libtmux

TMUX_SESSION_NAME = 'Fabric'

server = libtmux.Server()
sessions = server.sessions.filter(lambda x: x.session_name.startswith(TMUX_SESSION_NAME))
print(sessions)
if len(sessions) == 0:
    session = server.new_session(session_name=TMUX_SESSION_NAME)

[Session($1 Fabric), Session($2 Fabric1)]


In [44]:
import libtmux

TMUX_SESSION_NAME = 'Fabric'

server = libtmux.Server()
sessions = server.sessions.filter(lambda x: x.session_name.startswith(TMUX_SESSION_NAME))
#print(sessions)
if len(sessions) != 0:
    session = sessions[0]
    window = session.attached_window
    pane = window.attached_pane
    pane.send_keys('^B', enter=False)
    pane.send_keys('d', enter=False)
    

In [48]:
%%file ../fabric_example/fabfile/tmux.py
from fabric import task, Task
from invoke.watchers import Responder
import inspect
import libtmux

TMUX_SESSION_NAME = 'Fabric'

@task
def ConnectTmux(c):
    server = libtmux.Server()
    #session = server.sessions.get(session_name=TMUX_SESSION_NAME)
    session = server.sessions.filter(lambda x: x.session_name.startswith(TMUX_SESSION_NAME))
    if len(sessions) == 0:
        session = server.new_session(session_name=f'{TMUX_SESSION_NAME}')

def detach(c):
    detachResponder = Responder(
        pattern=r'^b^d',
        response='^Bd'
    )
    result = c.run(
        "^b^d",
        hide=False, pty=True,
        watchers=[detachResponder]
    )
    """
    server = libtmux.Server()
    sessions = server.sessions.filter(lambda x: x.session_name.startswith(TMUX_SESSION_NAME))
    #print(sessions)
    if len(sessions) != 0:
        session = sessions[0]
        window = session.attached_window
        pane = window.attached_pane
        pane.send_keys('^B', enter=False)
        pane.send_keys('d', enter=False)
    """

def tmux_run(*args, **kwargs):
    if len(args) == 1 and callable(args[0]):
        print(f'args: {args}, kwargs: {kwargs}')
        func = args[0]
        
        def inner(*args, **kwargs):
            sig = inspect.signature(func)
            keys = sig.parameters.keys()
            if len(keys) == 1:
                c = args[0]
                
                enterResponder = Responder(
                    pattern=r'\n>',
                    response='\n'
                )
                task_result = c.run(
                    "tmux a -t Fabric",
                    hide=False, pty=True,
                    watchers=[enterResponder]
                )
                
                func_result = func(*args)
            else:
                func_result = func(*args, **kwargs)
            print('===== detach')
            detach(c)
            return func_result
        
        inner.__name__ = func.__name__
        
        return inner


Overwriting ../fabric_example/fabfile/tmux.py
